In [1]:
import requests, pathlib
import os

# Chat model

In [2]:
# from langchain.chat_models import init_chat_model
# from dotenv import load_dotenv, find_dotenv

# dotenv_path = find_dotenv()
# load_dotenv(dotenv_path)

# model = init_chat_model("google_genai:gemini-2.5-flash-lite")

In [3]:
# Debug: check which key is loaded
gem_api = os.getenv("GOOGLE_API_KEY")
print(f"Key length: {len(gem_api) if gem_api else 0}")
print(f"Key starts with: {gem_api[:15] if gem_api else 'None'}...")
print(f"Key ends with: ...{gem_api[-5:] if gem_api else 'None'}")

Key length: 39
Key starts with: AIzaSyDX28fv_xQ...
Key ends with: ...EmrOA


In [4]:
type(gem_api)

str

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI
model = ChatGoogleGenerativeAI(
    google_api_key = os.getenv("GOOGLE_API_KEY"),
    model = "gemini-2.5-flash-lite",
)

/home/bip/archeion/langgraph-tutorial/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
model.invoke("In one line explain what is langgraph")

AIMessage(content='LangGraph is a Python library for building and deploying multi-agent applications and stateful workflows with LLMs.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--507ce16b-afe7-4bd9-83db-874793dbc7ed-0', usage_metadata={'input_tokens': 9, 'output_tokens': 22, 'total_tokens': 31, 'input_token_details': {'cache_read': 0}})

# Configuring Database

In [7]:
url = "https://storage.googleapis.com/benchmarks-artifacts/chinook/Chinook.db"
local_path = pathlib.Path("chinook.db")

In [8]:
if local_path.exists():
    print(f'{local_path} already exists. Skipping Download.')
else:
    response = requests.get(url)
    if response.status_code == 200:
        local_path.write_bytes(response.content)
        print(f'File downloaded and saved as {local_path}.')
    else:
        print(f'Failed to download file. Status code: {response.status_code}')

chinook.db already exists. Skipping Download.


In [9]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri(f"sqlite:///{local_path}")
print(f"Dialect: {db.dialect}")
print(f"Available Tables: {db.get_usable_table_names()}")
print(f"Sample Output: {db.run('SELECT * FROM artists LIMIT 5;')}")

Dialect: sqlite
Available Tables: ['albums', 'artists', 'customers', 'employees', 'genres', 'invoice_items', 'invoices', 'media_types', 'playlist_track', 'playlists', 'tracks']
Sample Output: [(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains')]


In [10]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db = db, llm = model)
tools = toolkit.get_tools()

for tool in tools:
    print(f"Tool name: {tool.name}, description: {tool.description}")

Tool name: sql_db_query, description: Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.
Tool name: sql_db_schema, description: Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3
Tool name: sql_db_list_tables, description: Input is an empty string, output is a comma-separated list of tables in the database.
Tool name: sql_db_query_checker, description: Use this tool to double check if your query is correct before executing it. Always use this tool before executing a query with sql_db_query!


In [11]:
from typing import Literal
from langchain.messages import AIMessage
from langchain_core.runnables import RunnableConfig
from langgraph.graph import END, START, MessagesState, StateGraph
from langgraph.prebuilt import ToolNode

In [ ]:
get_schema_tool = next(tool for tool in tools if tool.name == "sql_db_schema")
get_schema_node = ToolNode([get_schema_tool], name="Get DB Schema")